In [ ]:
%%capture
pip install cobra

In [2]:
import cobra
modelpath  = "/content/drive/MyDrive/Halomonas_Model_devt/HalomonasmodelV1.xml"
model1path = "/content/drive/MyDrive/Halomonas_Model_devt/HalomonasModelFBC.xml"

In [5]:
from pandas import DataFrame
import csv

In [3]:
model = cobra.io.read_sbml_model(modelpath)
model1 = cobra.io.read_sbml_model(model1path)

In [4]:
#Use this if there is no such a substrate file, but you want to use the original suggestions from carveme
media_file = model1.medium
Carbon_source = []
for i in media_file.keys():
  Carbon_source.append(i)
print(Carbon_source)

['EX_glc__D_e', 'EX_h2o_e', 'EX_14glucan_e', 'EX_h_e', 'EX_2hxmp_e', 'EX_2pglyc_e', 'EX_34dhbz_e', 'EX_34dhcinm_e', 'EX_leu__L_e', 'EX_3hoxpac_e', 'EX_3mb_e', 'EX_LalaDgluMdap_e', 'EX_4hbz_e', 'EX_4hoxpacd_e', 'EX_ala__L_e', 'EX_4hthr_e', 'EX_cl_e', 'EX_LalaDgluMdapDala_e', 'EX_5mtr_e', 'EX_dextrin_e', 'EX_arab__L_e', 'EX_4abut_e', 'EX_acac_e', 'EX_acald_e', 'EX_acgam1p_e', 'EX_acgam_e', 'EX_ac_e', 'EX_chol_e', 'EX_acmana_e', 'EX_acmum_e', 'EX_pi_e', 'EX_acser_e', 'EX_actn__R_e', 'EX_adn_e', 'EX_nh4_e', 'EX_ade_e', 'EX_agm_e', 'EX_mal__L_e', 'EX_alaala_e', 'EX_asn__L_e', 'EX_cys__L_e', 'EX_ala__D_e', 'EX_gln__L_e', 'EX_gly_e', 'EX_ser__L_e', 'EX_thr__L_e', 'EX_arg__L_e', 'EX_alltn_e', 'EX_cgly_e', 'EX_malt_e', 'EX_malttr_e', 'EX_anhgm_e', 'EX_arab__D_e', 'EX_fe3_e', 'EX_lys__L_e', 'EX_orn_e', 'EX_asp__L_e', 'EX_phe__L_e', 'EX_aso3_e', 'EX_aso4_e', 'EX_k_e', 'EX_pro__L_e', 'EX_meoh_e', 'EX_butso3_e', 'EX_but_e', 'EX_bz_e', 'EX_ca2_e', 'EX_15dap_e', 'EX_chols_e', 'EX_chtbs_e', 'EX_cit_e'

In [6]:
#Lets do a function that runs all the substrate components without having to put them individually
#I am creating a function that will take individual substrate elements and return the flux value.
def CarbonSource_with_SubstrateFile(model, SubstrateFile):
  """Simulates the effects of carbon sources on the objective of a flux balance solution.
  model is an xml file substratefile is a txt file with a list of substrates obtained from the initial draft reconstruction."""
  Simulation = {}
  #i = substrate_compound
  for i in SubstrateFile:
    model = cobra.io.read_sbml_model(modelpath)
    model.reactions.get_by_id('EX_glc__D_e').lower_bound = 0.0
    model.medium[i] = 1000
    model.reactions.get_by_id(i).lower_bound = -10.0
    solution = model.optimize()
    flux = solution.objective_value
    Simulation[i] = flux
    del model.medium[i]
    Medium_log = DataFrame.from_dict(model.medium, orient = 'index', columns = ["Carbon_source,  Upper_bound"])#just to control to make sure only one carbon compound was used at a time. This is the medium component at the time the last substrate was run
  Simulation_results = DataFrame.from_dict(Simulation, orient = 'index', columns = ["Carbon_source,  Flux_value"])
  return Simulation_results

In [7]:
Results = CarbonSource_with_SubstrateFile(model, Carbon_source)
Results

,"Carbon_source, Flux_value"
EX_glc__D_e,1.024067
EX_h2o_e,0.000000
EX_14glucan_e,6.105017
EX_h_e,0.000000
EX_2hxmp_e,0.000000
...,...
EX_udcpp_e,0.000000
EX_ura_e,0.000000
EX_xan_e,0.000000
EX_xtsn_e,0.000000


In [8]:
DataFrame.to_csv(Results,'/content/drive/MyDrive/Halomonas_Model_devt/CarbonSimulations.csv')

Ignnore the analysis below. was just trying to gapfill and some consistenc testing and blocked reaction testing

In [16]:
model

Name,M_Halomonas_annotation_feature_dna
Memory address,0x07fd9cb629350
Number of metabolites,1531
Number of reactions,2310
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"unknownCompartment1, unknownCompartment2, unknownCompartment3"


In [15]:
cobra.flux_analysis.find_blocked_reactions(model)

['14GLUCANabcpp',
 '14GLUCANtexi',
 '2HH24DDH',
 '2HXMPt6',
 '2PGLYCt6',
 '34DHCINMtex',
 '34DHCINMtpp',
 '3HOXPACt2pp',
 '3hoxpactex',
 '3HPAOX',
 '3MBt2pp',
 '3MBt4pp',
 '3MBtex',
 '4CMCOAS',
 '4H2KPILY',
 '4HOXPACDtex',
 '4HOXPACMOF',
 '4HOXPACt2pp',
 'AAMYL',
 'ABUTtex',
 'ACGAM1Ptex',
 'ACGApts',
 'ACGAptspp',
 'ACGAtex',
 'ACMANApts',
 'ACMANAptspp',
 'ACMANAt2',
 'ACMANAtex',
 'ACMUMptspp',
 'ACMUMtex',
 'ACTD',
 'ACTD2',
 'ACTDa',
 'ADNt2',
 'ALAGLUE',
 'ALCD3',
 'ALCD4',
 'ALDD1',
 'ALDD19xr',
 'ALDD31',
 'ALDD31_1',
 'ALDD4',
 'ALDD6',
 'AMANAPEr',
 'AMANK',
 'AMMQT8_2',
 'AMPTASEPG',
 'ARABDI',
 'ARAI',
 'ASO3t8pp',
 'ASO3tex',
 'ASO4rt2rpp',
 'ASO4tex',
 'ASR',
 'BETALDHx',
 'BETALDHy',
 'BHMT',
 'BMOCOS',
 'BMOGDS1',
 'BMOGDS2',
 'BTDD_RR',
 'BUTCT',
 'BUTSO3abcpp',
 'BUTSO3tex',
 'CACOAHA',
 'CAFFCOA',
 'CHOLD',
 'CHOLD2',
 'CHOLSabc',
 'CHOLSabc_1',
 'CHOLS_ex',
 'CHOLSH',
 'CHTBSptspp',
 'CHTBStex',
 'CMCMSAD',
 'CMHMI',
 'COCOAHA',
 'CPMPS',
 'CYNTAH',
 'CYNTt2',
 'CYN

In [9]:
#Lets try a gapfill
import cobra.test
from cobra.flux_analysis import gapfill

In [10]:
universal = cobra.Model("universal_reactions")

In [13]:
solution = gapfill(model, demand_reactions=False)
for reaction in solution[0]:
    print(reaction.id)

In [14]:
consistent_model = cobra.flux_analysis.fastcc(model)
consistent_model.reactions

[<Reaction 1PPDCRc at 0x7fd9c5f75e50>,
 <Reaction 2AGPE120tipp at 0x7fd9c5f75f10>,
 <Reaction 2AGPE140tipp at 0x7fd9c5f75490>,
 <Reaction 2AGPE141tipp at 0x7fd9c5f75350>,
 <Reaction 2AGPE160tipp at 0x7fd9c5f75310>,
 <Reaction 2AGPE161tipp at 0x7fd9c5f75c50>,
 <Reaction 2AGPE180tipp at 0x7fd9c5f75810>,
 <Reaction 2AGPE181tipp at 0x7fd9c5f756d0>,
 <Reaction 2AGPEAT120 at 0x7fd9c5f75610>,
 <Reaction 2AGPEAT140 at 0x7fd9c5f75450>,
 <Reaction 2AGPEAT141 at 0x7fd9c5f75050>,
 <Reaction 2AGPEAT160 at 0x7fd9c5f75290>,
 <Reaction 2AGPEAT161 at 0x7fd9c5f759d0>,
 <Reaction 2AGPEAT180 at 0x7fd9c5f75a10>,
 <Reaction 2AGPEAT181 at 0x7fd9c5f75b50>,
 <Reaction 2AGPG120tipp at 0x7fd9c5f75590>,
 <Reaction 2AGPG140tipp at 0x7fd9c5f75c10>,
 <Reaction 2AGPG141tipp at 0x7fd9c5f75150>,
 <Reaction 2AGPG160tipp at 0x7fd9c5f75190>,
 <Reaction 2AGPG161tipp at 0x7fd9c5f75110>,
 <Reaction 2AGPG180tipp at 0x7fd9c5f75210>,
 <Reaction 2AGPG181tipp at 0x7fd9c5f75690>,
 <Reaction 2AGPGAT120 at 0x7fd9c5f75990>,
 <Reactio

In [17]:
consistent_model

Name,M_Halomonas_annotation_feature_dna
Memory address,0x07fd9ca365710
Number of metabolites,1092
Number of reactions,1647
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"unknownCompartment1, unknownCompartment2, unknownCompartment3"


In [18]:
consistent_model.optimize()

,fluxes,reduced_costs
1PPDCRc,0.000000,-5.514209e-02
2AGPE120tipp,0.000000,4.993720e-16
2AGPE140tipp,0.000000,0.000000e+00
2AGPE141tipp,0.000000,0.000000e+00
2AGPE160tipp,0.000000,-5.691282e-16
...,...,...
sink_sheme_c,0.000000,0.000000e+00
Growth,1.024067,-4.718448e-16
ATPM,0.000000,-7.877441e-03
4HBZR,0.000099,1.249001e-16


In [19]:
cobra.io.write_sbml_model(consistent_model, "/content/drive/MyDrive/Halomonas_Model_devt/consistent_model.xml")